In [45]:
from problog.program import PrologString
from problog import get_evaluatable
import os# define the arguments for the model:

def get_model_files(date:str, model_name:str):
    model_files = []
    for root, dirs, files in os.walk(os.path.join("src/generated/models", date)):
        for file in files:
            if file.startswith("prolog_model_"+model_name+"_"):
                model_files.append(os.path.join(root, file))

    models_string_dir = {}
    for file in model_files:
        with open(file, 'r') as f:
            models_string_dir[file] = f.read()
    print(f"Found {len(models_string_dir)}")
    return models_string_dir

def get_single_model_file_with_path(model_path:str):

    if os.path.isfile(model_path):
        with open(model_path, 'r') as f:
            model_string = f.read()
    else:
        print(f"this file could not be found")
    return model_string

def prolog_reasoning(model:str):
    result = get_evaluatable().create_from(PrologString(model)).evaluate()
    print("% ProbLog Inference Result：")
    for query_key, probability in result.items():
        print(f"{query_key} = {probability:.4f}")

### Theme could be: World Travel
A person/drone travels through different countries, but he is not good at math,
He tries to solve math problems in the local language using traditional calculation methods

In [46]:
model = [""] * 10

#### Current Gregorian calendar date

In [47]:
model[0] = """
current_date(Year, Month, Day) :-
    Year  > 0,
    Month >= 1, Month =< 12,
    Day   >= 1, Day   =< 31.

gregorian_to_jdn(Y, M, D, JDN) :-
    A is (14 - M) // 12,
    Y1 is Y + 4800 - A,
    M1 is M + 12*A - 3,
    JDN is D
        + ((153*M1 + 2)//5)
        + 365*Y1
        + Y1//4
        - Y1//100
        + Y1//400
        - 32045.
"""

##### Bangla（孟加拉语）
Uses the "Bengali calendar" (বঙ্গাব্দ), which has fixed date conversion rules with the Gregorian calendar.

In [48]:
model[1] = """
bangla_date(GY,GM,GD,BY,BM,BD) :-
    ( GM > 4 ; GM =:= 4, GD >= 14 ),    % on or after April 14
    BY is GY - 593,
    base_and_today_jdn(GY,GM,GD, J0, J),
    Offset is J - J0,
    bangla_month_day(Offset, BM, BD).

bangla_date(GY,GM,GD,BY,BM,BD) :-
    ( GM < 4 ; GM =:= 4, GD < 14 ),     % before April 14
    BY is GY - 594,
    base_and_today_jdn(GY,GM,GD, J0, J),
    Offset is J - J0,
    bangla_month_day(Offset, BM, BD).

base_and_today_jdn(GY,GM,GD, J0, J) :-
    gregorian_to_jdn(GY, 4, 14, J0),
    gregorian_to_jdn(GY, GM, GD, J).

bangla_month_day(Offset, BM, BD) :-
    Offset < 31*5,
    BM is Offset // 31 + 1,
    BD is Offset mod 31 + 1.
bangla_month_day(Offset, BM, BD) :-
    Offset >= 31*5,
    Off2 is Offset - 31*5,
    BM is Off2 // 30 + 6,
    BD is Off2 mod 30 + 1.
"""

##### Devanagari（天城文，常见于印地语等）
Indian languages ​​(such as Hindi, Marathi, etc.) often use the Hindu calendar (Vikram Samvat, Shaka Samvat, etc.), which is a lunisolar calendar and needs to be converted according to the moon phase and the position of the sun.

In [49]:
model[2] = """
shaka_date(GY,GM,GD,SY,SM,SD) :-
    ( GM > 2 ; GM =:= 2, GD >= 22 ),
    SY is GY - 78, SM = GM, SD = GD.
shaka_date(GY,GM,GD,SY,SM,SD) :-
    ( GM < 2 ; GM =:= 2, GD < 22 ),
    SY is GY - 79, SM = GM, SD = GD.
"""

##### Arabic（阿拉伯语）
Uses the Islamic calendar (Hijri), a pure lunar calendar, with 29 or 30 days per month, and a year that differs from the Gregorian calendar by about 11 days.


##### Urdu（乌尔都语）
In the Muslim community, religious festivals are mostly counted according to the Islamic lunar calendar (Hijri).

In [50]:
model[3] = """
hijri_date(GY,GM,GD,HY,HM,HD) :-
    gregorian_to_jdn(GY, GM, GD, J),
    Days is J - 1948439,    % Hijri epoch JDN(1 Muharram,1AH)
    HY   is (30*Days + 10646) // 10631,
    Hijri1 is (HY-1)*354 + ((3 + 11*(HY-1))//30) + 1948439,
    DOY  is J - Hijri1 + 1,
    hijri_month_day(DOY, 1, HM, HD).

% 月长：奇月30天，偶月29天
month_length(M, 30) :-  1 is M mod 2.
month_length(M, 29) :-  0 is M mod 2.

hijri_month_day(Day, M, M, Day) :-
    month_length(M, L),
    Day =< L.
hijri_month_day(Day, M, HM, HD) :-
    month_length(M, L),
    Day > L,
    Day1 is Day - L,
    M1 is M + 1,
    hijri_month_day(Day1, M1, HM, HD).
"""

##### Farsi（波斯语）
Uses the Persian solar calendar (Solar Hijri), with the spring equinox as the beginning of the year, and needs to be accurately converted with the Gregorian calendar.

In [51]:
model[4] = """
% 判断闰年
leap_persian(Y) :-
    Rem is (Y*682) mod 2816,
    Rem < 682.

% 月/日分解：前186天31天/月，后180天30天/月
persian_month_day(DOY, PM, PD) :-
    DOY =< 186,
    PM is ((DOY - 1) // 31) + 1,
    PD is ((DOY - 1) mod 31) + 1.
persian_month_day(DOY, PM, PD) :-
    DOY > 186,
    Off is DOY - 186,
    PM is ((Off - 1) // 30) + 7,
    PD is ((Off - 1) mod 30) + 1.

% 计算上一波斯年总天数（365 或 366）
prev_year_length(PY, Length) :-
    Prev is PY - 1,
    leap_persian(Prev),
    Length is 366.
prev_year_length(PY, Length) :-
    Prev is PY - 1,
    \+ leap_persian(Prev),
    Length is 365.

% 波斯太阳历主谓词：春分 JNew 后
persian_date(GY,GM,GD,PY,PM,PD) :-
    gregorian_to_jdn(GY,GM,GD,J),
    EpBase is GY - 474,
    EpYear is 474 + (EpBase mod 2820),
    March1 is 365*EpYear + EpYear//4 - EpYear//128 + EpYear//2820,
    JNew   is March1 + 226895 + (31*(EpYear*682 - 110))//2816,
    J >= JNew,
    PY is EpBase mod 2820 + 1,
    J1 is JNew,
    DOY is J - J1 + 1,
    persian_month_day(DOY, PM, PD).

% 波斯太阳历主谓词：春分 JNew 前
persian_date(GY,GM,GD,PY,PM,PD) :-
    gregorian_to_jdn(GY,GM,GD,J),
    EpBase is GY - 474,
    EpYear is 474 + (EpBase mod 2820),
    March1 is 365*EpYear + EpYear//4 - EpYear//128 + EpYear//2820,
    JNew   is March1 + 226895 + (31*(EpYear*682 - 110))//2816,
    J < JNew,
    PY is EpBase mod 2820,
    prev_year_length(PY, TotalDays),
    J1 is JNew - TotalDays,
    DOY is J - J1 + 1,
    persian_month_day(DOY, PM, PD).

"""

##### Telugu（泰卢固语）
Uses the Telugu calendar, which is also based on the Hindu lunisolar calendar system.

##### Kannada（卡纳达语）
Uses the Kannada calendar, which is also a lunisolar calendar of India.

##### Tibetan（藏语）
Uses the Tibetan calendar, which is based on the lunar cycle and supplemented by the solar year, and has its own set of intercalary month and leap month rules.

Not achievable, requires astronomical algorithms

In [52]:
model[5] = """"""
model[6] = """"""
model[7] = """"""

In [53]:
queries = """
query(bangla_date(2025, 5, 20, BY, BM, BD)).
query(shaka_date(2025, 5, 20, SY, SM, SD)).
query(hijri_date(2025,5,20,HY,HM,HD)).
query(persian_date(2025,5,20,PY,PM,PD)).
"""
model_str = "\n".join(model) + queries

In [ ]:
prolog_reasoning(model_str)

% ProbLog Inference Result：
bangla_date(2025,5,20,1432,2,6) = 1.0000
shaka_date(2025,5,20,1947,5,20) = 1.0000
hijri_date(2025,5,20,1446,11,24) = 1.0000
persian_date(2025,5,20,1552,49304,8) = 1.0000
